In [ ]:
import os
from pathlib import Path

import numpy as np
import xarray as xr
import xesmf as xe

from oslofjord_sim.regrid import (
    fill_diagonal_pairs,
    fill_secondary_diagonal_pairs,
    fill_surrounded_nans,
    replace_surrounded_values,
)

### The script prepares a bathymetry for FjordsSim to construct a grid.
This is a latitude - longitude grid with equal geographical coordinates spacing.
First, choose the required grid z face levels.
We use a ROMS grid file for the Oslofjord as a basis.

In [ ]:
Z_FACES = np.array(
    [
        -450.0,
        -400.0,
        -350.0,
        -300.0,
        -250.0,
        -200.0,
        -150.0,
        -100.0,
        -75.0,
        -50.0,
        -25.0,
        -15.0,
        -10.0,
        -7.5,
        -5.0,
        -3.0,
        -2.0,
        -1.0,
        0.0,
    ]
)
INPUT_GRID = os.path.join(Path.home(), "FjordSim_data", "oslofjord", "OF160_grid_v2.nc")
ds_in = xr.open_dataset(INPUT_GRID)

Original grid:

In [ ]:
h = ds_in.h * ds_in.mask_rho
h.plot()

In [ ]:
print(f"Lat min: {ds_in.lat_rho.values.min()}")
print(f"Lat max: {ds_in.lat_rho.values.max()}")
print(f"Lon min: {ds_in.lon_rho.values.min()}")
print(f"Lon max: {ds_in.lon_rho.values.max()}")

Chose the area, spacing and then regrid using XESMF

In [ ]:
lat = np.linspace(59.0, 59.93, num=232)
lon = np.linspace(10.2, 11.02, num=105)

In [ ]:
ds_out = xr.Dataset(
    {
        "lat": (["lat"], lat, {"units": "degrees_north"}),
        "lon": (["lon"], lon, {"units": "degrees_east"}),
    }
)
regridder = xe.Regridder(ds_in.rename({"lon_rho": "lon", "lat_rho": "lat"}), ds_out, "bilinear", unmapped_to_nan=True)
da_h_orig = regridder(h)

In [ ]:
da_h = da_h_orig.where(da_h_orig > 0, np.nan)  # replace h=0 with nans
da_h = da_h.clip(min=10)  # and set a minimum depth to 10 meters
da_h.plot(figsize=(7, 10))

Fix grid

In [ ]:
da_h.values = fill_secondary_diagonal_pairs(fill_diagonal_pairs(da_h.values))
da_h.plot(figsize=(7, 10))

In [ ]:
for _ in range(10):
    da_h.values = fill_surrounded_nans(replace_surrounded_values(da_h.values))  # run several times
da_h.plot(figsize=(7, 10))

Save grid

In [ ]:
z_faces_da = xr.DataArray(
    Z_FACES,
    dims=("z_faces",),
    coords={"z_faces": Z_FACES},
    name="z_faces",
)

In [ ]:
da_h.values = -1 * da_h.values
da_h = da_h.where(da_h < 0, 10)
dsout = xr.Dataset(
    {
        "h": da_h,
        "z_faces": z_faces_da,
    }
)

In [ ]:
dsout

In [ ]:
dsout.to_netcdf(os.path.join(Path.home(), "FjordSim_data", "oslofjord", "bathymetry_105to232.nc"))